In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
from deriva.chisel import Model, Schema, Table, Column, Key, ForeignKey, builtin_types
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "362" #@param
# host = 'www.eye-ai.org'
host = 'dev.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-10-13 19:07:46,800 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-13 19:07:46,801 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-10-13 19:07:46,803 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:139679141727008]
2024-10-13 19:07:46,803 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2024-10-13 19:07:46,804 - INFO - RenewingAuthorizer will start by using access_token with hash "02bb75540d43deea45c213e0222c306aabe82bd169c476ce94c4ab4d445478ed"
2024-10-13 19:07:46,804 - INFO - Executing token refresh without client credentials
2024-10-13 19:07:46,805 - INFO - Fetching new token from Globus Auth
2024-10-13 19:07:47,207 - INFO - request done (success)
2024-10-13 19:07:47,208 - INFO - RenewingAuthorizer.

You are already logged in.


In [4]:

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.
EA = EyeAI(hostname = host, catalog_id = catalog_id, 
           cache_dir= cache_dir, working_dir=working_dir)

2024-10-13 19:07:52,893 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-13 19:07:52,894 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [5]:
cv_list = {}
for t in EA.find_vocabularies():
    cv_list[t.name] = {'schema': t.schema.name, 'fks': t.referenced_by}

full_list = cv_list
full_list

{'Diagnosis_Subject_Vocab': {'schema': 'eye-ai', 'fks': []},
 'Diagnosis_Observation_Vocab': {'schema': 'eye-ai', 'fks': []},
 'Diagnosis_Image_Vocab': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Diagnosis_Diagnosis_Vocab_fkey' at 0x7f09951f8970>]},
 'Image_Side_Vocab': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Image_Image_Side_Vocab_fkey' at 0x7f0995a027d0>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Clinical_Records_Image_Side_Vocab_fkey' at 0x7f0995213f40>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'OCR_RNFL_Image_Side_Vocab_fkey' at 0x7f099503c940>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'OCR_HVF_Image_Side_Vocab_fkey' at 0x7f099503e170>]},
 'Image_Angle_Vocab': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Image_Image_Angle_Vocab_fkey' at 0x7f0995a02830>]},
 'Image_Quality_Vocab': {'schema': 'eye-ai',
  'fks':

In [22]:
# del cv_list['Diagnosis_Subject_Vocab']
# del cv_list['Diagnosis_Subject_Vocab_Vocab']
# del cv_list['Diagnosis_Observation_Vocab']
# del cv_list['Diagnosis_Observation_Vocab_Vocab']

# del cv_list['Execution_Metadata_Type']
# del cv_list['Workflow_Type_vocab']
# del cv_list['Execution_Asset_Type_Vocab']

# del cv_list['Diagnosis_Image_Vocab']
del cv_list['Diagnosis_Image_Vocab_Vocab']
# del cv_list['Image_Angle_Vocab']
# del cv_list['Image_Side_Vocab']
del cv_list['Image_Side_Vocab_Vocab']

# del cv_list['Image_Quality_Vocab']
# del cv_list['Subject_Gender']
# del cv_list['Subject_Ethnicity']
# del cv_list['Subject_hypertension']
# del cv_list['Subject_insulin_dependent']
# del cv_list['Subject_pregnant']
# del cv_list['Subject_cataract']
# del cv_list['Subject_maculopathy']
# del cv_list['Subject_other']
# del cv_list['Subject_image_quality']
# del cv_list['Subject_image_quality_factor']
# del cv_list['Image_Output']


cv_list

{'Diagnosis_Tag': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Diagnosis_Diagnosis_Tag_fkey' at 0x7f09951f8a30>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Subject_Diagnosis_Diagnosis_Tag_fkey' at 0x7f09951fabf0>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Observation_Diagnosis_Diagnosis_Tag_fkey' at 0x7f09951fb880>]},
 'Diagnosis_Status': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Diagnosis_Diagnosis_Status_fkey' at 0x7f09951f8a90>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Subject_Diagnosis_Diagnosis_Status_fkey' at 0x7f09951fac20>,
   <deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Observation_Diagnosis_Diagnosis_Status_fkey' at 0x7f09951fb8e0>]},
 'Image_Tag': {'schema': 'eye-ai',
  'fks': [<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Image_Image_Tag_fkey' at 0x7f0995a02950>]},
 'Process_Tag': {'schema': 'eye-ai',
  'fks': [<deriva.core.

In [13]:
def create_new_cv_table(cv_schema_name, old_cv_table_name, t_name):
    cv_schema = EA.catalog.getPathBuilder().schemas[cv_schema_name]
    
    # retrieve exiting terms
    old_cv_table = cv_schema.tables[old_cv_table_name]
    path = old_cv_table.path
    entities = pd.DataFrame(path.entities())
    terms_ingest = entities[['Name', 'Description', 'Synonyms']]
    terms_ingest_mapping = entities[['RID', 'Name']]
    
    # create new table
    if(t_name not in cv_schema.tables):
        new_vocab_table = EA.create_vocabulary(
            vocab_name = t_name,
            schema= cv_schema_name)

        # ingest exiting terms
        ingest_list = terms_ingest.to_dict(orient='records')
        cv_schema = EA.catalog.getPathBuilder().schemas[cv_schema_name]
        new_vocab = cv_schema.tables[t_name]
        new_vocab.insert(ingest_list, defaults={'ID', 'URI'})
    return terms_ingest_mapping

def update(table, mapping_list, num_up):
    n = len(mapping_list)
    for i in range(n // num_up):
        table.update(mapping_list[i * num_up: (i + 1) * num_up], [table.RID])
        logging.info(f"Updated indices: {i * num_up} to {(i + 1) * num_up}")
    remaining_start = (i + 1) * num_up if n >= num_up else 0
    if remaining_start < n:
        table.update(mapping_list[remaining_start: n], [table.RID])
        logging.info(f"Updated indices: {remaining_start} to {n}")


def build_association(cv_schema_name, old_cv_table_name, asso_schema_name, asso_table_name, 
                      t_name, fk_col_name, terms_ingest_mapping):
    EA = EyeAI(hostname = host, catalog_id = catalog_id, 
           cache_dir= cache_dir, working_dir=working_dir)
    cv_model_schema = EA.model.schemas[cv_schema_name]
    asso_model_schema = EA.model.schemas[asso_schema_name]
    
    # 3. build FK
    asso_table = asso_model_schema.tables[asso_table_name]
    new_vocab_table = cv_model_schema.tables[t_name]
    try:
        asso_table.create_reference((t_name, True, new_vocab_table))
    except:
        pass

    # 4. add new references - Name
    asso_schema = EA.catalog.getPathBuilder().schemas[asso_schema_name]
    asso_entities = asso_schema.tables[asso_table_name]
    path = asso_entities.path

    terms_ingest = pd.DataFrame(path.entities())[['RID', fk_col_name]]
    terms_ingest = pd.merge(terms_ingest, terms_ingest_mapping, how='left', left_on=fk_col_name, right_on='RID')
    
    mapping_ingest = terms_ingest[['RID_x', 'Name']]
    mapping_ingest.rename(columns={'RID_x':'RID', 'Name':t_name}, inplace=True)

    mapping_ingest.dropna(inplace=True)
    mapping_list = mapping_ingest.to_dict(orient='records')
    update(asso_entities, mapping_list, 500)
    # asso_entities.update(mapping_list, [asso_entities.RID])
    

In [7]:
def refactor_cv_tables(cv_table_dict):
    for key, values in cv_table_dict.items():
        old_cv_table_name = key
        logging.info(f"Updating cv table {key}")
        t_name = old_cv_table_name+'_Vocab'
        cv_schema_name = values['schema']
        # create new cv table with values:
        terms_ingest_mapping = create_new_cv_table(cv_schema_name, old_cv_table_name, t_name)
        
        # find associated tables
        for fk in values['fks']:
            fk_cols = []
            for col in fk.columns:
                asso_schema_name = col.table.schema.name
                asso_table_name = col.table.name
                fk_col_name = col.name
                # Build association and update FK col
                build_association(cv_schema_name, old_cv_table_name, 
                                  asso_schema_name, asso_table_name, 
                                  t_name, fk_col_name,
                                  terms_ingest_mapping)
                fk_cols.append(col)
            fk.drop()
            for c in fk_cols:
                c.drop()

In [23]:
# test_list = {}
# test_list['Image_Side_Vocab'] = cv_list['Image_Side_Vocab']

refactor_cv_tables(cv_list)
test_list

2024-10-13 20:38:42,861 - INFO - Updating cv table Diagnosis_Tag
2024-10-13 20:38:43,858 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-10-13 20:38:43,859 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
/tmp/ipykernel_19320/932850689.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mapping_ingest.rename(columns={'RID_x':'RID', 'Name':t_name}, inplace=True)
/tmp/ipykernel_19320/932850689.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

{'Image_Side_Vocab': {'schema': 'eye-ai', 'fks': []}}

In [35]:
def cleanup_table_names():
    non_vocab_keys = {}
    vocab_vocab_keys = {}
    vocab_keys = {}
    
    for t in EA.find_vocabularies():
        if not t.name.endswith('_Vocab'):
            non_vocab_keys[t.name] = {'schema': t.schema.name, 'fks': t.referenced_by}
        elif t.name.endswith('_Vocab_Vocab'):
            vocab_vocab_keys[t.name] = {'schema': t.schema.name, 'fks': t.referenced_by}
        else:
            vocab_keys[t.name] = {'schema': t.schema.name, 'fks': t.referenced_by}

    # Drop old cv tables
    for key, values in non_vocab_keys.items():
        for fk in values['fks']:
            print(fk)
            fk.drop()
        table = EA.model.schemas[values['schema']].tables[key]
        table.drop()

    # Drop old cv table and change _Vocab_Vocab table name to _Vocab
    for key, values in vocab_vocab_keys.items():
        old_cv_table_name = key[:-6]
        for fk in vocab_keys[old_cv_table_name]['fks']:
            fk.drop()
        old_cv_table = EA.model.schemas[values['schema']].tables[old_cv_table_name]
        old_cv_table.drop()
    
        cv_table = EA.model.schemas[values['schema']].tables[key]
        cv_table.alter(
            schema_name=values['schema'],
            table_name=old_cv_table_name)

In [47]:
# cleanup_table_names()

<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_Gender' at 0x7f09951c1d20>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_Ethnicity' at 0x7f09951c2830>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_hypertension' at 0x7f09951c3340>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_insulin_dependent' at 0x7f09951c3e50>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_pregnant' at 0x7f09951d49a0>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_cataract' at 0x7f09951d54b0>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_maculopathy' at 0x7f09951d5fc0>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_other' at 0x7f09951d6ad0>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_image_quality' at 0x7f09951d75e0>
<deriva.core.ermrest_model.ForeignKey object 'eye-ai'.'Subject_image.._Subject_image_quality_factor_fkey_g12gJ' at 0x7f09951e1ea0>
<deriva.core.ermrest_model.Table object 'eye-ai'.'Subject_image_quali

In [57]:
for key, values in vocab_vocab_keys.items():
    old_cv_table_name = key[:-6]
    print(old_cv_table_name)
    for fk in vocab_keys[old_cv_table_name]['fks']:
        fk.drop()
    old_cv_table = EA.model.schemas[values['schema']].tables[old_cv_table_name]
    old_cv_table.drop()

    cv_table = EA.model.schemas[values['schema']].tables[key]
    cv_table.alter(
        schema_name=values['schema'],
        table_name=old_cv_table_name)


Diagnosis_Image_Vocab
Image_Side_Vocab
Diagnosis_Subject_Vocab
Diagnosis_Observation_Vocab
Image_Angle_Vocab
Image_Quality_Vocab
